-----
<div> <center> ESPACIO PARA BANNER DE LA MAESTRIA </center> </div>   

-----

# Clusteres espaciales

El contenido de este notebook es tomado del siguiente [enlace](https://darribas.org/gds_scipy16/). Para más información visitar las referencias

Nuevamente usaremos los datos de AirBnb en Austin, pero esta vez nos adentraremos en el tópico de regresión espacial.

La premisa básica de los ejercicios que haremos en este cuaderno es que a través de las características de las casas listadas en AirBnb podemos aprender sobre la geografía de Austin. En concreto, intentaremos clasificar los códigos postales de la ciudad en un número reducido de grupos que nos permitan extraer algunos patrones sobre los principales tipos de casas y zonas de la ciudad.

In [2]:
pip install pysal

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Carguemos unas librerías iniciales

%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps
import geopandas as gpd
from sklearn import cluster
from sklearn.preprocessing import scale

sns.set(style="whitegrid")

ModuleNotFoundError: No module named 'pysal'

## Datos

Definimos el directorio de nuestros datos. Esta línea debe ajustarla para la ruta particular de su computador

In [ ]:
listings_link = './data/listings.csv.gz'

# Referencias
- Rey, S., & Arribas-Bel, D. (nd). Introduction Geographic Data Science with PySAL and the pydata stack